# ML modelling with MLOps
#### Split data by browser and platform_os, and version each split as a new version of the data in dvc.


In [4]:
# Importing Pandas an Numpy Libraries to use on manipulating our Data
import pandas as pd
import numpy as np

In [5]:
# To Preproccesing our data
from sklearn.preprocessing import LabelEncoder

In [6]:
# To fill missing values
from sklearn.impute import SimpleImputer

In [7]:
# To Split our train data
from sklearn.model_selection import train_test_split

In [8]:
# To Visualize Data
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# To Train our data
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB

In [10]:
# To evaluate end result we have
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score